In [9]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model

# Cargar el modelo entrenado
model = load_model('model.h5')

# Captura desde la webcam
cap = cv2.VideoCapture(0)

# Lista de clases
class_names = [str(i) for i in range(10)]

# ROI
x, y, w, h = 150, 100, 200, 200

# Variables para animación de barrida
scan_line_y = y
direction = 1  # 1 = abajo, -1 = arriba

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    roi = gray[y:y+h, x:x+w]

    roi_resized = cv2.resize(roi, (32, 32))
    roi_normalized = roi_resized.astype("float32") / 255.0
    roi_input = roi_normalized.reshape(1, 32, 32, 1)

    predictions = model.predict(roi_input, verbose=0)[0]
    predicted_class = np.argmax(predictions)
    confidence = predictions[predicted_class] * 100

    output_frame = frame.copy()

    # Dibujar el ROI
    cv2.rectangle(output_frame, (x, y), (x + w, y + h), (255, 255, 0), 2)

    # Animación de barrida luminosa (línea blanca semitransparente)
    overlay = output_frame.copy()
    cv2.line(overlay, (x, scan_line_y), (x + w, scan_line_y), (255, 255, 255), 2)
    alpha = 0.3
    output_frame = cv2.addWeighted(overlay, alpha, output_frame, 1 - alpha, 0)

    # Actualizar posición de la línea
    scan_line_y += direction * 5
    if scan_line_y >= y + h or scan_line_y <= y:
        direction *= -1

    # Mostrar todas las probabilidades
    for i, prob in enumerate(predictions):
        text = f"{class_names[i]}: {prob:.2f}"
        color = (0, 255, 0) if i == predicted_class else (0, 0, 255)
        cv2.putText(output_frame, text, (10, 30 + i * 25),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)

    # Mensaje dinámico debajo del ROI
    mensaje_confianza = f"Tu numero es un {predicted_class} ({confidence:.1f}%)"
    mensaje_indicacion = "Posiciona el numero dentro del recuadro"

    # Centrar texto respecto a la ventana, no al ROI
    (frame_height, frame_width) = output_frame.shape[:2]
    (text_width, _), _ = cv2.getTextSize(mensaje_indicacion, cv2.FONT_HERSHEY_DUPLEX, 0.8, 2)
    text_x = (frame_width - text_width) // 2
    text_y = y + h + 35

    # Mostrar mensaje según confianza
    if confidence < 95:
        if int(time.time() * 2) % 2 == 0:
            cv2.putText(output_frame, mensaje_indicacion, (text_x, text_y),
                        cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 140, 255), 2)
    else:
        (text_width_c, _), _ = cv2.getTextSize(mensaje_confianza, cv2.FONT_HERSHEY_DUPLEX, 0.8, 2)
        text_x_c = (frame_width - text_width_c) // 2
        cv2.putText(output_frame, mensaje_confianza, (text_x_c, text_y),
                    cv2.FONT_HERSHEY_DUPLEX, 0.8, (0, 255, 0), 2)

    # Mostrar imagen original + probabilidades
    cv2.imshow("Imagen original + probabilidades", output_frame)

    # Mostrar imagen al modelo (ampliada para mejor visibilidad)
    roi_display = cv2.resize(roi_resized, (128, 128), interpolation=cv2.INTER_NEAREST)
    cv2.imshow("Imagen al modelo", roi_display)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()